<a href="https://colab.research.google.com/github/Evora-21805468/IA-2021/blob/main/Aula3_LAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3
> Turing Machine 

## Purpose
Practical introduction to Turing Machine

## Methodology  
A step-by-step implementation of a Turing Machine example

## Work in Progress 

- 2021-03-07 19:33:38 - Lab 3

## Results
A functional implementation of a Turing Machine example

## Suggested next steps
Complete all the exercises

## Setup

### Library import

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

## Inputer Program Exercise

Let's imagine a simple blank tape as the one below, where each cell can store a value.  
We want to create a program that stores the value ```111```on the tape.

![image.png](attachment:image.png)

### Start

Imagine a pointer named ```header``` pointing at a cell position on the tape. In the example below the header is represented by the wider black square.

![image.png](attachment:image.png)

### Program  

1. If the value on the cell is blank, store a value and move right.

First iteration:  
- write

![image.png](attachment:image.png)

- move

![image.png](attachment:image.png)

- repeat ...

#### Program summary

| Read  | Write Instruction |  Move Instruction |
|---|---|---|
| Blank  |  1  | R  | 

#### State

We need to stop eventually otherwise the program will never end and will keep writing `1` on each cell of the tape.

| State | Read  | Write Instruction |  Move Instruction | New State|
|---|---|---| --- | --- | 
| 0 | Blank  |  1  | R  | 1 |
| 1 | Blank  |  1  | R  | 2 |
| 2 | Blank  |  1  | R  | Halt |

![image.png](attachment:image.png)

### Implementation

Let's use a list to simulate our blank tape

In [ ]:
# create an empy list
tape = []
tape

[]

Our tape will have 10 Blank cells

In [ ]:
# our definition of blank
blank = 'blank'

for _ in range(10):
    tape.append(blank)
    
tape

['blank',
 'blank',
 'blank',
 'blank',
 'blank',
 'blank',
 'blank',
 'blank',
 'blank',
 'blank']

#### 1. We need to read the tape's cell values given the ```header``` position

In [ ]:
def read_tape(position, tape):
    """
    Reads tape at current position
    """

    # Non-disruptive
    #if position < 0 or position >= len(tape):
    #  return -1

    # Disruptive
    #if position <0 or position >= len(tape):
    #   raise Exception("position out of range")

    assert position >= 0 and position < len(tape),\
    "position out of tape's range"
    
    return tape[position]

In [ ]:
read_tape(1,tape)

'blank'

#### 2. Write a function to obtain the instructions and the the next stage 

To define such a function we need the program, the current state and the value that was read from the tape.  
Recall the program

| State | Read  | Write Instruction |  Move Instruction | New State|
|---|---|---| --- | --- | 
| 0 | Blank  |  1  | R  | 1 |
| 1 | Blank  |  1  | R  | 2 |
| 2 | Blank  |  1  | R  | Halt |

##### 2.1 Create a Pandas DataFrame to define the program

In [ ]:
program =\
pd.DataFrame(
[
 {
     'State': '0',
      'Read': 'blank',
      'Write_instruction': '1',
      'Move_instruction': 'R',
      'New_State': '1',
 },
 {
     'State': '1',
      'Read': 'blank',
      'Write_instruction': '1',
      'Move_instruction': 'R',
      'New_State': '2',
 },
 {
     'State': '2',
      'Read': 'blank',
      'Write_instruction': '1',
      'Move_instruction': 'R',
      'New_State': 'Halt',
 }
]
)

program

,State,Read,Write_instruction,Move_instruction,New_State
0,0,blank,1,R,1
1,1,blank,1,R,2
2,2,blank,1,R,Halt


In [ ]:
def get_instruction(program, state, symbol_read):
    """
    Get the program instructions given the given state and symbol input
    """
    
    # identify state and current input on the program
    current_state_mask = program['State'] == state
    symbol_read_mask = program['Read'] == symbol_read

    mask = current_state_mask & symbol_read_mask
    assert mask.sum() == 1, "Invalid instructions" 
    
    features = ['Write_instruction', 'Move_instruction', 'New_State']
    
    return program[mask][features]

Example

In [ ]:
get_instruction(program, '1', 'blank')

AssertionError: ignored

#### 3. Assign variables to each instruction and to the new state

In [ ]:
write_instruction = get_instruction(program, '1', 'blank')['Write_instruction']
write_instruction

AssertionError: ignored

In [ ]:
type(write_instruction)

pandas.core.series.Series

Get the scalar value

In [ ]:
write_instruction.iat[0]

'1'

In [ ]:
write_instruction.iloc[0]

'1'

Do the same for the Move instruction and the next stage

We can also extract a Numpy MultiDim-Array from the Pandas DataFrame 

In [ ]:
get_instruction(program, '1', 'blank')

,Write_instruction,Move_instruction,New_State
1,1,R,2


In [ ]:
array = get_instruction(program, '1', 'blank').values
array

array([['1', 'R', '2']], dtype=object)

In [ ]:
array[0]

array(['1', 'R', '2'], dtype=object)

#### 4. Define a function to update the tape

Given a tape, which is of type list, define a function to update the list given an index

In [ ]:
def update_tape(tape, position, value):
    """
    Updates tape at current position with given value
    """
    
    tape[position] = value
    
    return tape

#### 5. Define a function to update the ```header```

The move on the tape can only be Left or Right ('L' or 'R'). Define a function that updates the ```header```position accordingly.

In [ ]:
def update_head(position, move, tape_size):
    """
    Update head position
    """

    # Compute next position
    if move == 'L':
       next_position = position - 1
    elif move == 'R':
        next_position = position + 1

    # Validation
    assert next_position >= 0 and next_position < tape_size,\
    "Invalid head position"

    return next_position

### Compile every step to a single function

#### 6.  Define a function that given a program, a tape, the ```header```and a program state computes the following:
1. Reads the value on the tape given the ```header````
2. Obtain the program instructions given the current state and the symbol read
3. Update the tape
5. Update the ```header````
6. Return the new ```header``` position, the next state, and the updated tape

In [ ]:
def execute_step(program, tape, head, state) :
    """
    Executes a single step of the turing machine
    """
    initial_msg = "\t\t# Initial State: {}".format(state)
    print(initial_msg)
    
    # Read tape
    symbol_read = read_tape(head, tape)
    
    # Load instructions according to reading
    instructions = get_instruction(program, state, symbol_read)
    
    # Unfold instructions
    write_instruction, move_instruction, next_state = instructions.values[0]
    
    # Update tape
    tape = update_tape(tape, head, write_instruction)
    
    # Update head
    tape_size = len(tape)
    next_position = update_head(head, move_instruction, tape_size)
    
    return next_position, next_state, tape

### Almost Done! 

#### 7. Define a function to execute the program 

In [ ]:
def run_machine(program, tape, head=0, state='0'):
    """
    The main function
    """

    iterator = 1
    while state != 'Halt':

        print("""## Iteration : {}
        # Tape  : {}
        # Head is at position : {}""".format(iterator,
                                             tape,
                                             head))

        # Execute Step
        next_position, next_state, tape = execute_step(program, tape, head, state)

        # Updates
        head = next_position
        state = next_state
        iterator += 1
        
    print('## Program Finished')
    return tape

In [ ]:
program

,State,Read,Write_instruction,Move_instruction,New_State
0,0,blank,1,R,1
1,1,blank,1,R,2
2,2,blank,1,R,Halt


In [ ]:
tape = ['blank' for _ in range(10)]
run_machine(program, tape)

## Iteration : 1
        # Tape  : ['blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
        # Head is at position : 0
## Iteration : 2
        # Tape  : ['1', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
        # Head is at position : 1
## Iteration : 3
        # Tape  : ['1', '1', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
        # Head is at position : 2
## Program Finished


['1', '1', '1', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']

### Exercícios

### Define um Programa que devolva a seguinte tape


```python
['blank', 'blank', 'blank', 'blank', '1', '1', '1', 'blank', 'blank', 'blank']
```


In [ ]:
run_machine(program, tape, head=4)

## Iteration : 1
        # Tape  : ['1', '1', '1', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
        # Head is at position : 4
## Iteration : 2
        # Tape  : ['1', '1', '1', 'blank', '1', 'blank', 'blank', 'blank', 'blank', 'blank']
        # Head is at position : 5
## Iteration : 3
        # Tape  : ['1', '1', '1', 'blank', '1', '1', 'blank', 'blank', 'blank', 'blank']
        # Head is at position : 6
## Program Finished


['1', '1', '1', 'blank', '1', '1', '1', 'blank', 'blank', 'blank']

### Define um Programa que altere todos os digitos consecutivos ```1``` em uma tape para ```0``` ignorando ```blank```


Exemplo, se a tape possuir o seguinte 
```python
['blank', '1', '1', '1', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
```
  
deve retornar
```python
['blank', '0', '0', '0', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
```

In [ ]:
program =\
pd.DataFrame(
[
    {
      'State': '0',
      'Read': 'blank',
      'Write_instruction': 'blank',
      'Move_instruction': 'R',
      'New_State': '0', 
    },
    {
      'State': '0',
      'Read': '1',
      'Write_instruction': '0',
      'Move_instruction': 'R',
      'New_State': '1', 
    },
    {
      'State': '1',
      'Read': '1',
      'Write_instruction': '0',
      'Move_instruction': 'R',
      'New_State': '1', 
    },
    {
      'State': '0',
      'Read': 'blank',
      'Write_instruction': 'blank',
      'Move_instruction': 'R',
      'New_State': 'Halt', 
    },
]
)

program

,State,Read,Write_instruction,Move_instruction,New_State
0,0,blank,blank,R,0
1,0,1,0,R,1
2,1,1,0,R,1
3,0,blank,blank,R,Halt


In [ ]:
tape = ['blank', '1', '1', '1', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
run_machine(program, tape, 0)

## Iteration : 1
        # Tape  : ['blank', '1', '1', '1', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
        # Head is at position : 0
		# Initial State: 0


AssertionError: ignored

### Define um Programa que some ```1``` a um número presente numa tape  

Exemplo, se a tape possuir o seguinte 
```python
['blank', '1', '1', '1', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
```
  
deve retornar
```python
['blank', '1', '1', '2', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
```

In [ ]:
program =\
pd.DataFrame(
[
    # your code here
]
)

program

In [ ]:
tape = ['blank', '1', '1', '9', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
run_machine(program, tape)

## Iteration : 1
        # Tape  : ['blank', '1', '1', '9', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
        # Head is at position : 0
## Iteration : 2
        # Tape  : ['blank', '1', '1', '9', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
        # Head is at position : 1
## Iteration : 3
        # Tape  : ['blank', '1', '1', '9', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
        # Head is at position : 2
## Iteration : 4
        # Tape  : ['blank', '1', '1', '9', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
        # Head is at position : 3
## Iteration : 5
        # Tape  : ['blank', '1', '1', '9', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
        # Head is at position : 4
## Iteration : 6
        # Tape  : ['blank', '1', '1', '9', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank', 'blank']
        # Head is at position : 3
## Iteration : 7
        # Tape  : ['blank', '1', '1', '0'

['blank',
 '1',
 '2',
 '0',
 'blank',
 'blank',
 'blank',
 'blank',
 'blank',
 'blank',
 'blank']